# learn about simulation

In [7]:
class test:
    def __init__(self) -> None:
        
        self.__a =1
        self.b =2

    def apply_a(self,n):
        self.__a=n


In [8]:
asr = test()

In [5]:
asr._a = 3

In [6]:
asr._a

3

In [1]:
import numpy as np


In [2]:
np.finfo(np.float32).min

-3.4028235e+38

In [3]:
np.finfo(np.float32).max

3.4028235e+38

In [74]:
from collections import deque
f=1
s=0
histo = deque(maxlen=f*(s+1)-s)
init_item = [np.array([1,i],np.float32) for i in range(f*(s+1)+1)]
histo.extend(init_item)
sampling = [histo[i] for i in range(len(histo)) if i%(s+1)==0]
histo,sampling

(deque([array([1., 1.], dtype=float32)], maxlen=1),
 [array([1., 1.], dtype=float32)])

In [76]:
histo.append(np.array([1,43]))
sampling = [histo[i] for i in range(len(histo)) if i%(s+1)==0]
histo,sampling

(deque([array([ 1, 43])], maxlen=1), [array([ 1, 43])])

In [78]:
np.concatenate(sampling)

array([ 1, 43])

In [80]:
def tes(**arg):
    print(arg)

In [81]:
tes(img =1, act = 3)

{'img': 1, 'act': 3}


In [4]:
from gym import spaces

observation_space = spaces.Box(low=np.finfo(np.float32).min,
                                                high=np.finfo(np.float32).max,
                                                shape=(1, 7),
                                                dtype=np.float32)

In [6]:
observation_space.sample()

array([[ 2.4290013e+38,  9.9281949e+37, -1.4790196e+38, -2.5282738e+38,
         2.6497033e+38, -1.1968979e+38,  1.9680949e+38]], dtype=float32)

In [7]:
np.repeat(observation_space.high, 3, axis=-1)

array([[3.4028235e+38, 3.4028235e+38, 3.4028235e+38, 3.4028235e+38,
        3.4028235e+38, 3.4028235e+38, 3.4028235e+38, 3.4028235e+38,
        3.4028235e+38, 3.4028235e+38, 3.4028235e+38, 3.4028235e+38,
        3.4028235e+38, 3.4028235e+38, 3.4028235e+38, 3.4028235e+38,
        3.4028235e+38, 3.4028235e+38, 3.4028235e+38, 3.4028235e+38,
        3.4028235e+38]], dtype=float32)

In [9]:
obs = spaces.Box(low=0, high=255,shape=(10,12,3), dtype=np.uint8)

In [11]:
obs.sample().shape

(10, 12, 3)

In [ ]:
from pygame.locals import K_TAB

In [3]:
import numpy as np
from collections import deque
from gym import spaces

In [54]:
observation_space = spaces.Dict({
            'seg': spaces.Box(low=0, high=3, shape=(16,16,4), dtype=np.uint8)
        })

In [62]:
action_space = spaces.Box(np.array([-1, 0]), np.array([1, 1]), dtype=np.float32) 

/home/lpr/miniconda3/envs/CarlaEnv/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [63]:
observation_space.spaces['action'] = action_space

In [66]:
stacked_action_space = spaces.Tuple((action_space,) * 4)

In [67]:
stacked_action_space

Tuple(Box([-1.  0.], 1.0, (2,), float32), Box([-1.  0.], 1.0, (2,), float32), Box([-1.  0.], 1.0, (2,), float32), Box([-1.  0.], 1.0, (2,), float32))

In [6]:
import pygame
import cv2
import os


In [2]:
def set_world(world):
    settings = world.get_settings()
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 0.2
    settings.max_substeps = 16
    settings.max_substep_delta_time = 0.0125
    world.apply_settings(settings)

In [3]:
def setting_camera(cam_type,cam_config):
        
    bp_cam = blueprints.find(cam_type)
    bp_cam.set_attribute('image_size_y', str(cam_config['image_size_y']))
    bp_cam.set_attribute('image_size_x', str(cam_config['image_size_x']))
    bp_cam.set_attribute('fov', str(cam_config['fov']))
    bp_cam.set_attribute('sensor_tick', str(cam_config['sensor_tick']))

    return bp_cam

In [3]:
from config.position import car_spawn
from utils.tools import carla_point
from config.camera import front_cam,left_cam,right_cam

import carla
carSPs = [carla_point(p) for p in car_spawn]

camera_list = [front_cam,left_cam,right_cam]
camera_dict = {s['name']:s for s in camera_list}

# Connect to the CARLA server
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

# Get the version information
version = client.get_client_version()
print("CARLA Version:", version)

world = client.get_world()

set_world(world)

# Destroy all actors
actors = world.get_actors()
for actor in actors:
    actor.destroy()

blueprints = world.get_blueprint_library()

bp_car = blueprints.filter("evt_echo_4s")[0]

car = world.spawn_actor(bp_car, carSPs[0])

cams = []

for s in camera_list:
    bp_cam = setting_camera('sensor.camera.semantic_segmentation',s)
    cam = world.spawn_actor(bp_cam, carla.Transform(carla.Location(*s['Location']), carla.Rotation(*s['Rotation'])), attach_to=car)
    dir = os.path.join("save",s['name'])
    os.makedirs(dir,exist_ok=True)
    cam.listen(lambda image,dir = dir: image.save_to_disk(os.path.join(dir,str(len(os.listdir(dir))+1)+".jpg")))
    cams.append(cam)

pygame.init()
pygame.font.init()
display = pygame.display.set_mode((1120, 560), pygame.HWSURFACE | pygame.DOUBLEBUF)
clock = pygame.time.Clock()

CARLA Version: 0.9.15


In [ ]:
def process_image(image):
    img = pygame.image.frombuffer(image.raw_data, (image.width, image.height), 'RGB')
    display.blit(img, (0, 0))

In [9]:
import keyboard

In [3]:
import pygame
import time
# Initialize Pygame
pygame.init()

# Set up the display
pygame.display.set_caption("test carla")
clock = pygame.time.Clock()

# Load an image
image = pygame.image.load("save/test1.jpg")

# Main loop
running = True
st=time.time()
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Clear the screen
    display.fill((255, 255, 255))
    
    # Draw the image onto the screen
    display.blit(image, (100, 100))  # Drawing the image at (100, 100) on the screen

    # Update the display
    pygame.display.flip()

    clock.tick()
    world.tick()

# Quit Pygame
pygame.quit()


In [7]:
# Example file showing a basic pygame "game loop"
import pygame

# pygame setup
pygame.init()
screen = pygame.display.set_mode((1280, 720))
clock = pygame.time.Clock()
running = True

while running:
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # fill the screen with a color to wipe away anything from last frame
    screen.fill("purple")

    # RENDER YOUR GAME HERE

    # flip() the display to put your work on screen
    pygame.display.flip()

    clock.tick(60)  # limits FPS to 60

pygame.quit()

In [ ]:

pygame.event.pump()
if pygame.key.get_pressed()[K_ESCAPE]:
    pygame.close()

clock.tick()


In [27]:
while True:
    world.tick()

KeyboardInterrupt: 

In [20]:
# save image
name = "test1"
cams[int(name[-1])].listen(lambda image,name=name: image.save_to_disk(f'save/{name}.jpg'))

In [21]:
cams[int(name[-1])].stop()

In [24]:
# Destroy actors
cams[int(name[-1])].destroy()


True

In [94]:
control = carla.VehicleAckermannControl(steer=0, steer_speed=0.0 ,speed=0.0, acceleration=0, jerk=0.0)
car.apply_ackermann_control(control)

In [85]:
control = carla.VehicleControl(throttle=0, steer=-1, brake=0,
                                hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
car.apply_control(control)

In [95]:
car.destroy()

True

In [ ]:
world.tick()

In [13]:
import carla
from config.position import car_spawn
from utils.tools import carla_point




In [97]:
from gym import spaces
import numpy as np

In [98]:
action_space = spaces.Box(np.array([-1, 0]), np.array([1, 1]), dtype=np.float32)

/home/lpr/miniconda3/envs/CarlaEnv/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [102]:
a,b = action_space.sample()
a,b

(0.5843601, 0.17324854)

In [25]:


observation_space = spaces.Dict({
            'seg': spaces.Box(low=0, high=255, shape=(10,15,10 ,20), dtype=np.uint8),
            'action': spaces.Box(low=-1.0, high=1.0, shape=(8,), dtype=np.float32),
        })


In [33]:
observation_space.sample()['seg'].shape

(10, 15, 10, 20)

In [20]:
random_image = np.random.randint(0, 256, size=(10, 20, 3), dtype=np.uint8)

In [23]:
list_img = [random_image]*5

In [28]:
a= np.array(list_img, dtype=np.uint8)
a.shape

(5, 10, 20, 3)

In [30]:
state_seg_tmp = np.concatenate((a, a), axis=0)

In [31]:
state_seg_tmp.shape

(10, 10, 20, 3)

In [45]:
from config.camera import front_cam,right_cam,left_cam

camera_list = [front_cam,right_cam,left_cam]
camera_list

[{'image_size_x': 483,
  'image_size_y': 272,
  'fov': 69,
  'sensor_tick': 0.001,
  'name': 'front_camera',
  'Location': (0.98, 0, 1.675),
  'Rotation': (-12.5, 0, 0)},
 {'image_size_x': 483,
  'image_size_y': 272,
  'fov': 69,
  'sensor_tick': 0.001,
  'name': 'right_camera',
  'Location': (0, 0.61, 1.675),
  'Rotation': (-30, 90, 0)},
 {'image_size_x': 483,
  'image_size_y': 272,
  'fov': 69,
  'sensor_tick': 0.001,
  'name': 'left_camera',
  'Location': (0, -0.61, 1.675),
  'Rotation': (-30, -90, 0)}]

In [ ]:
{cam['name']:spaces.Box(low=0, high=255, shape=(8,SEG_SIZE,), dtype=np.uint8) for cam in camera_list}

In [ ]:
observation_space = spaces.Dict({
            'seg': spaces.Box(low=0, high=255, shape=(IN_CHANNLES,SEG_SIZE,), dtype=np.uint8),
            'action': spaces.Box(low=-1.0, high=1.0, shape=(N_LOOK_BACK,), dtype=np.float32),

        })

In [27]:
a =spaces.Box(np.array([-1, 0]), np.array([1, 1]))

In [28]:
a.dtype

dtype('float32')

In [26]:
action_space = spaces.Box(np.array([-1, 0]), np.array([1, 1]), dtype=np.float32)

/home/lpr/miniconda3/envs/CarlaEnv/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [48]:
action_space.sample()

array([0.30563948, 0.8410887 ], dtype=float32)